# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7fe8e425e0>
├── 'a' --> tensor([[ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452]])
└── 'x' --> <FastTreeValue 0x7f7febed2ee0>
    └── 'c' --> tensor([[ 0.2562,  0.6296,  0.4366,  1.4298],
                        [-0.4975,  0.6626,  0.4765, -0.3334],
                        [-0.0507,  0.5951, -0.6898,  0.0171]])

In [4]:
t.a

tensor([[ 0.2876,  0.8974, -1.5416],
        [ 0.8404,  1.2731,  0.3452]])

In [5]:
%timeit t.a

58.3 ns ± 0.0352 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7fe8e425e0>
├── 'a' --> tensor([[ 1.1592, -0.4335,  0.5207],
│                   [-1.6381, -1.6067, -1.3390]])
└── 'x' --> <FastTreeValue 0x7f7febed2ee0>
    └── 'c' --> tensor([[ 0.2562,  0.6296,  0.4366,  1.4298],
                        [-0.4975,  0.6626,  0.4765, -0.3334],
                        [-0.0507,  0.5951, -0.6898,  0.0171]])

In [7]:
%timeit t.a = new_value

63.1 ns ± 0.0587 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2876,  0.8974, -1.5416],
               [ 0.8404,  1.2731,  0.3452]]),
    x: Batch(
           c: tensor([[ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2876,  0.8974, -1.5416],
        [ 0.8404,  1.2731,  0.3452]])

In [11]:
%timeit b.a

56.1 ns ± 0.0218 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4959, -0.5470,  0.7071],
               [ 1.6819,  0.0499, -1.1525]]),
    x: Batch(
           c: tensor([[ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.179 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 0.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 876 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 847 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7fe8e3b370>
├── 'a' --> tensor([[[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]],
│           
│                   [[ 0.2876,  0.8974, -1.5416],
│                    [ 0.8404,  1.2731,  0.3452]]])
└── 'x' --> <FastTreeValue 0x7f7f1c016760>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 53.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7fe8d93550>
├── 'a' --> tensor([[ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452],
│                   [ 0.2876,  0.8974, -1.5416],
│                   [ 0.8404,  1.2731,  0.3452]])
└── 'x' --> <FastTreeValue 0x7f7febe571f0>
    └── 'c' --> tensor([[ 0.2562,  0.6296,  0.4366,  1.4298],
                        [-0.4975,  0.6626,  0.4765, -0.3334],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 58.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.3 µs ± 87.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2562,  0.6296,  0.4366,  1.4298],
                       [-0.4975,  0.6626,  0.4765, -0.3334],
                       [-0.0507,  0.5951, -0.6898,  0.0171]],
              
                      [[ 0.2562,  0.6296,  0.4366,  1.4298],
                       [-0.4975,  0.6626,  0.4765, -0.3334],
                       [-0.0507,  0.5951, -0.6898,  0.0171]],
              
                      [[ 0.2562,  0.6296,  0.4366,  1.4298],
                       [-0.4975,  0.6626,  0.4765, -0.3334],
                       [-0.0507,  0.5951, -0.6898,  0.0171]],
              
                      [[ 0.2562,  0.6296,  0.4366,  1.4298],
                       [-0.4975,  0.6626,  0.4765, -0.3334],
                       [-0.0507,  0.5951, -0.6898,  0.0171]],
              
                      [[ 0.2562,  0.6296,  0.4366,  1.4298],
                       [-0.4975,  0.6626,  0.4765, -0.3334],
                       [-0.0507,  0.5951, -0.6898,  0.0171]],

In [26]:
%timeit Batch.stack(batches)

80 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171],
                      [ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171],
                      [ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171],
                      [ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171],
                      [ 0.2562,  0.6296,  0.4366,  1.4298],
                      [-0.4975,  0.6626,  0.4765, -0.3334],
                      [-0.0507,  0.5951, -0.6898,  0.0171],
                      [ 0.2562,  0.6296,  0.4366,  1.4298],
                   

In [28]:
%timeit Batch.cat(batches)

150 µs ± 404 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 2.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
